In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import numpy as np
import cv2
import mediapipe as mp

# ===================================================================
# BƯỚC 1: ĐỊNH NGHĨA LẠI CÁC HÀM VÀ CLASS GIỐNG HỆT KHI HUẤN LUYỆN
# ===================================================================

# --- Định nghĩa kiến trúc mô hình ---
MAX_FRAMES = 128
RESIZE_TO = (128, 128)

class VideoClassifier3DCNN(nn.Module):
    def __init__(self, num_classes):
        super(VideoClassifier3DCNN, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv3d(3, 16, kernel_size=(3, 3, 3), padding=1), nn.ReLU(),
            nn.BatchNorm3d(16), nn.MaxPool3d(kernel_size=(2, 2, 2)))
        self.conv_block2 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), padding=1), nn.ReLU(),
            nn.BatchNorm3d(32), nn.MaxPool3d(kernel_size=(2, 2, 2)))
        self.conv_block3 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=1), nn.ReLU(),
            nn.BatchNorm3d(64), nn.MaxPool3d(kernel_size=(2, 2, 2)))
        fc_input_dim = 64 * (MAX_FRAMES // 8) * (RESIZE_TO[0] // 8) * (RESIZE_TO[1] // 8)
        self.classifier = nn.Sequential(
            nn.Linear(fc_input_dim, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, num_classes))

    def forward(self, x):
        x = x.permute(0, 2, 1, 3, 4)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# --- Hàm MỚI: Vẽ Skeleton đè lên Video gốc ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def create_overlay_video_from_path(video_path):
    """
    Đọc video, trích xuất skeleton và vẽ nó TRỰC TIẾP LÊN CÁC FRAME GỐC.
    Trả về một tensor PyTorch có shape (num_frames, height, width, 3).
    """
    holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(video_path)
    
    video_frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Tạo một bản sao của frame để vẽ lên, tránh sửa đổi frame gốc
        frame_to_draw = frame.copy()
        
        # Chuyển màu và xử lý bằng MediaPipe
        image_rgb = cv2.cvtColor(frame_to_draw, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)
        
        # Vẽ skeleton đè lên frame
        mp_drawing.draw_landmarks(
            frame_to_draw, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,0,0), thickness=1))
        mp_drawing.draw_landmarks(
            frame_to_draw, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=1))
        mp_drawing.draw_landmarks(
            frame_to_draw, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=1))
        
        # Chuyển frame BGR (của OpenCV) sang RGB để lưu vào tensor
        frame_rgb_output = cv2.cvtColor(frame_to_draw, cv2.COLOR_BGR2RGB)
        video_frames.append(frame_rgb_output)

    cap.release()
    holistic.close()
    
    if not video_frames:
        return torch.empty(0)
        
    return torch.from_numpy(np.array(video_frames))


# --- Hàm tiền xử lý cuối cùng (padding/normalize) ---
def preprocess_single_video(video_tensor):
    num_frames = video_tensor.shape[0]
    if num_frames > MAX_FRAMES: video_tensor = video_tensor[:MAX_FRAMES]
    elif num_frames < MAX_FRAMES:
        padding = torch.zeros((MAX_FRAMES - num_frames,) + video_tensor.shape[1:], dtype=video_tensor.dtype)
        video_tensor = torch.cat([video_tensor, padding], dim=0)

    transform = T.Compose([
        T.ToPILImage(),
        T.Resize(RESIZE_TO),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    processed_frames = torch.stack([transform(frame.numpy()) for frame in video_tensor])
    return processed_frames


# ===================================================================
# BƯỚC 2: THỰC HIỆN DỰ ĐOÁN
# ===================================================================
if __name__ == '__main__':
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MODEL_PATH = "sign_language_3dcnn_final.pth" 
    VIDEO_PATH = "output_hello_accurate.mp4"

    CLASS_NAMES = [ 'again', 'bad', 'bathroom', 'book', 'busy', 'do not want', 'eat', 'father', 'fine', 'finish', 'forget', 'go', 'good', 'happy', 'hello', 'help', 'how', 'i', 'learn', 'like', 'meet', 'milk', 'more', 'mother', 'my', 'name', 'need', 'nice', 'no', 'please', 'question', 'right', 'sad', 'same', 'see you letter', 'thank you', 'want', 'what', 'when', 'where', 'which', 'who', 'why', 'wrong', 'yes', 'you', 'your' ]
    NUM_CLASSES = len(CLASS_NAMES)

    # --- Tải mô hình ---
    print("Đang tải mô hình...")
    model = VideoClassifier3DCNN(num_classes=NUM_CLASSES)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    print("Tải mô hình thành công!")

    # --- QUY TRÌNH DỰ ĐOÁN ĐÚNG ---
    print(f"\n1. Vẽ skeleton đè lên video '{VIDEO_PATH}'...")
    overlay_video_tensor = create_overlay_video_from_path(VIDEO_PATH)

    if overlay_video_tensor.shape[0] == 0:
        print("Lỗi: Không thể xử lý video đầu vào.")
    else:
        print("2. Tiền xử lý video đã có skeleton...")
        processed_tensor = preprocess_single_video(overlay_video_tensor)
        

        input_tensor = processed_tensor.unsqueeze(0).to(DEVICE)
        print(f"3. Kích thước input cuối cùng: {input_tensor.shape}")
        
        print("4. Bắt đầu dự đoán...")
        with torch.no_grad():
            output = model(input_tensor)
            _, predicted_idx = torch.max(output, 1)
            predicted_class_index = predicted_idx.cpu().item()
            predicted_class_name = CLASS_NAMES[predicted_class_index]

        print("\n🚀 KẾT QUẢ DỰ ĐOÁN:")
        print(f"==> Chỉ số lớp dự đoán: {predicted_class_index}")
        print(f"==> Tên lớp dự đoán: {predicted_class_name}")

Đang tải mô hình...
Tải mô hình thành công!

1. Vẽ skeleton đè lên video 'output_hello_accurate.mp4'...
2. Tiền xử lý video đã có skeleton...
3. Kích thước input cuối cùng: torch.Size([1, 128, 3, 128, 128])
4. Bắt đầu dự đoán...

🚀 KẾT QUẢ DỰ ĐOÁN:
==> Chỉ số lớp dự đoán: 35
==> Tên lớp dự đoán: thank you
